# 최종학습데이터셋_부산_Shift&Merge

### import

In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import statistics
import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime as dt 
import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

### global

In [ ]:
## 예측 대상 지역
loc = "부산"

## 데이터셋 shift 일수
shift = 60

### 전처리 데이터셋 input

In [ ]:
## 전처리셋
가뭄지수 = pd.read_csv("../data/result/입력/가뭄지수_최종정리.csv", index_col = 0)
면적생산 = pd.read_csv("../data/result/입력/면적생산_최종정리.csv", index_col = 0)
일반유가 = pd.read_csv("../data/result/입력/일반유가_최종정리.csv", index_col = 0)
면세유가 = pd.read_csv("../data/result/입력/면세유가_최종정리.csv", index_col = 0)
수입량 = pd.read_csv("../data/result/입력/수입량_최종정리.csv", index_col = 0)
#도매가격 = pd.read_csv(f"../data/final/도매_가격_완성본_13_21년도_재수정.csv", index_col = 0)

## 한국행정구역분류표
addr = pd.read_excel('../data/지역/한국행정구역분류_2022.7.1.기준_20220630022706.xlsx', sheet_name = 1, header = [2, 2])
addr.drop('Unnamed: 0_level_0', axis = 1, inplace = True)
addr['시군구'].replace('예산시', '예산군')
addr.columns = addr.columns.droplevel(1)

## 21년도 도매가격 데이터
도매가격 = pd.read_csv(f"../data/도매가/보간/13_21_{loc}_도매가보간.csv", index_col = 0)

### 데이터셋 input (최종학습데이터)

In [ ]:
data = pd.read_csv(f'../data/result/입력/학습데이터1차_{loc}.csv', index_col = 0)

### Shift

In [ ]:
df_shift = copy.deepcopy(data)

In [ ]:
## 연도 컬럼 추가
def helper(x) :
    if (pd.isna(x)) :
        return x
    else :
        return x[:4]

df_shift['도매가격품명'] = ''
df_shift['도매가격날짜'] = ''
df_shift['도매가격연도'] = ''
df_shift['도매가격품명'] = df_shift['품명']
df_shift['도매가격날짜'] = df_shift['날짜'].apply(lambda x : str(x))
df_shift['도매가격연도'] = df_shift['도매가격날짜'].apply(lambda x : helper(x))

df_shift = pd.concat([df_shift.loc[:, '날짜' : 'EDI'], df_shift.loc[:, '도매가격' : '도매가격연도'].shift(-1 * shift)], axis = 1)

### 21년 데이터 concat.

In [ ]:
df_domae = 도매가격.drop('지역', axis = 1)
df_concat = df_shift.iloc[:, -4 :]

In [ ]:
df_concat_droped = df_concat.dropna()

In [ ]:
## 연도 컬럼 추가
def helper(x) :
    if (pd.isna(x)) :
        return x
    else :
        return x[:4]

df_domae.columns = ['도매가격날짜', '도매가격품명', '도매가격']
df_domae = df_domae[['도매가격', '도매가격품명', '도매가격날짜']]
df_domae['도매가격연도'] = df_domae['도매가격날짜'].apply(lambda x : helper(x))

In [ ]:
## 기존 데이터셋에 없던 21년 이후 데이터를 추가
idx = 도매가격[도매가격['날짜'] == '2021-01-01'].index
df_domae = df_domae.iloc[idx[0] : idx[0] + shift, :]

In [ ]:
df_concat_final = pd.concat([df_concat_droped, df_domae]).reset_index(drop = True)
df_concat_final

In [ ]:
df_shift_concat = copy.deepcopy(df_shift)

In [ ]:
## 컬럼 정리
df_shift_concat.drop(columns = ['도매가격', '도매가격품명', '도매가격날짜', '도매가격연도'], axis = 1, inplace = True)
df_shift_concat = pd.concat([df_shift_concat, df_concat_final], axis = 1)

### 면적생산 merge

In [ ]:
df_area = copy.deepcopy(df_shift_concat)

In [ ]:
면적생산['연도'] = 면적생산['연도'].apply(lambda x : str(x))

In [ ]:
# 연도 컬럼 추가
def helper(x) :
    if (pd.isna(x)) :
        return x
    else :
        return x[:4]


df_area['기존연도'] = df_area['날짜'].apply(lambda x : helper(x))

In [ ]:
df_area = pd.merge(df_area, 면적생산, left_on = ['기존연도', '도매가격품명'], right_on = ['연도', '품명'], how = 'left')
df_area

### 일반유가 merge

In [ ]:
df_taxoil = copy.deepcopy(df_area)

### 가뭄지수 관측지점의 시도명 추출 및 컬럼 추가

In [ ]:
## 시도명 컬럼 추가 (time...)
df_taxoil['시도명'] = ''

## 시도명 탐색 후 대입
def helper(target) :
    if(pd.isna(target)) :
        return target
    
    try :
        ans = addr[addr['시군구'] == target].iloc[0]['시도']
    except :
        ans = addr[addr['시도'] == target].iloc[0]['시도']
        
    
        
    if ans[:2] in 면세유가['지역'].unique() :
        return ans[:2]
    else :
        return (ans[0] + ans[2])

df_taxoil['시도명'] = df_taxoil['지명_y'].apply(lambda x : helper(x))

In [ ]:
df_taxoil = pd.merge(df_taxoil, 일반유가, left_on = ['날짜', '시도명'], right_on = ['날짜', '지역'], how = 'left')
df_taxoil

### 면세유가 merge

In [ ]:
df_dutyfreeoil = copy.deepcopy(df_taxoil)

In [ ]:
df_dutyfreeoil = pd.merge(df_dutyfreeoil, 면세유가, left_on = ['날짜', '시도명'], right_on = ['날짜', '지역'], how = 'left')
df_dutyfreeoil

### 수입량 merge

In [ ]:
df_import = copy.deepcopy(df_dutyfreeoil)

In [ ]:
# 수입량 시점 전처리
def helper(x) :
    if (pd.isna(x)) :
        return x
    
    td = relativedelta(months = 1)
    newDate = pd.to_datetime(x) - td
    
    y = newDate.year
    m = newDate.month

    if (m % 10 == m) :
        m = '0' + str(m)
    else :
        pass

    return (str(y) + '-' + str(m))

# 도매가격연월 컬럼 추가
df_import['도매가격연월'] = df_import['도매가격날짜'].apply(lambda x : helper(x))

In [ ]:
df_import = pd.merge(df_import, 수입량, left_on = '도매가격연월', right_on = '연월', how = 'left')
df_import

### 가뭄지수 merge

In [ ]:
df_drought = copy.deepcopy(df_import)

In [ ]:
df_drought = pd.merge(df_drought, 가뭄지수, left_on = ['날짜', '관측지점명_y'], right_on = ['발표일자', '시군명'], how = 'left')
df_drought

### 컬럼 정리

In [ ]:
df_columns = copy.deepcopy(df_drought)

In [ ]:
# drop 제외 대상
protected = ['날짜', '도매가격', '도매가격품명', '도매가격날짜', '지명_y', '면적_y', '생산량_y', 
             '관측지점명_y', '시도명_y', '일반유가_y', '면세유가_y', '수입중량_y', 'SPI1_y',  'SPI3_y', 'EDI_y']

In [ ]:
li = []
for item in list(df_columns.columns) :
    if not item in protected :
        li.append(item)
        
df_columns = df_columns.drop(columns = li)
df_columns

In [ ]:
df_columns.columns = ['날짜', '도매가격', '도매가격품명', '도매가격날짜', '지명', '면적', '생산량', '관측지점명',
                 '일반유가', '면세유가', '수입중량(30일전)', 'SPI1', 'SPI3', 'EDI', '시도명']
df_columns = df_columns[['날짜', '도매가격품명', '지명', '시도명', '관측지점명', '면적', '생산량', 'SPI1', 'SPI3', 'EDI', 
               '일반유가', '면세유가', '수입중량(30일전)', '도매가격날짜', '도매가격']]
df_columns

### 최종데이터셋 파일 저장

In [ ]:
loc

In [ ]:
df_result = copy.deepcopy(df_columns)
df_result.info()

In [ ]:
df_result.to_csv(f"../data/result/출력/최종학습데이터_{loc}.csv", encoding = 'utf-8-sig')